# Creating the Initial Commitment Transaction

In this section, we'll build a lightning channel initial commitment transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the basepoints derivated in the `chapter 0 - lightning node keys derivation`, and the funding transaction created in the `chapter 1 - channel funding transaction`.

In [1]:
# run notebook

%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 1 - channel funding transaction/funding-transaction.ipynb"

2025-09-20T00:21:32.183000Z TestFramework (INFO): PRNG seed is: 4912066725343081636
2025-09-20T00:21:32.184000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_ia1pggng
To obscure commitment number 0xd6ff4d17c776
Alice funding pubkey: 6b9446873f732ad65256122f549f68e5a65380c6114ab6866a78625c3a1db38e
Alice funding privkey: 4c71a9bebac1d2df85b7392418a214e1d2c924613dafaaaa2238f707dd29f421
Alice funding address: bcrt1pdw2ydpelwv4dv5jkzgh4f8mgukn98qxxz99tdpn20p39cwsakw8qg8wudq
Bob funding pubkey: 08d9ba1711457aa852c0a447947110453a334716f147e4db2f31e3b3fad2129e
Bob funding privkey: 125b800098723e395048a73fe2ea8dafb7c48142986474236979ba5f40d74562
Bob funding address: bcrt1pprvm59c3g4a2s5kq53regugsg5arx3ck79r7fke0x83m87kjz20q0xp4rn
Transaction creating Alice UTXO: 8d33e46e03766da1623decc3e9c3ba995a6d26af081b0218b39d5f840976be05
{
  "txid": "8d33e46e03766da1623decc3e9c3ba995a6d26af081b0218b39d5f840976be05",
  "hash": "8c59f4689d1847e2ef5896ea042b8b2cd0b0970414ae2e7ac68f9

At this point, no HTLCs have been added yet, which makes the initial commitment transaction simpler. Only after Alice has a fully signed initial commitment transaction she will broadcast the funding transaction, this serves as a guarantee for Alice (the funder) that she can reclaim her sats back if anything goes wrong.

### The Unsigned Transaction

### The Input

The input is the channel funding transaction.

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number on the opening channel 
commitment_number = 0
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

### The Outputs

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

Since this is Alice’s first commitment transaction, it will have no outputs to Bob. That’s because Alice is the funder of the channel and isn't sending any sats to Bob initially. If she wanted to transfer funds at channel opening, she could have set the `push_msat` field in the `open_channel` message, specifying the amount in millisatoshis.

Additionally, because the channel hasn’t been opened yet, there are no offered or received HTLCs — so no HTLC outputs will be created either.

As a result, the outputs in the first commitment transaction will be as follows:

Alice first Commitment Transaction will have two outputs:
* to_local_anchor_output
* to_local_output

Bob first Commitment Transaction will have two outputs:
* to_remote_anchor_output
* to_remote_output

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) specifies that transaction outputs must be sorted by value, from smallest to largest. Because of this, the to_local_anchor_output appears first — as it has a fixed amount of 330 sats, which corresponds to the default dust limit for P2WSH outputs.

**Anchor outputs** exists to prevent a malicious peer from attaching low-fee child transactions, which could block the victim from getting the commitment transaction confirmed in time. This defense was made possible by a change introduced in Bitcoin Core 0.19: [bitcoin/bitcoin#15681](https://github.com/bitcoin/bitcoin/pull/15681). It’s also the reason why all non-anchor outputs in the commitment transaction are CSV (CheckSequenceVerify)-locked: to delay their spendability and allow time for fee bumping via the anchor. The anchor outputs feature is optional and can be enabled only if both channel peers support it. However, starting with [Eclair v0.11.0](https://github.com/ACINQ/eclair/blob/master/docs/release-notes/eclair-v0.11.0.md) stop accepting channels that don't support anchor outputs.

A **trimmed HTLC** is an HTLC that does not get included in the commitment transaction because its value is too low to be economically spent. Specifically, if the value of the HTLC is below the dust limit plus the estimated fee required to claim it, the output is trimmed — in other words, left out of the transaction entirely.

This mechanism helps avoid bloating the commitment transaction with outputs that would either be unspendable or cost more in fees than they're worth.


### Alice First Commitment Transaction Outputs

#### To Local Anchor Output

This is the output that the local party, will be able to use to CPFP the commitment transaction and it is spendable by anyone after 16 blocks.

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------------------+
                    ---| P(local_delayed) + T |
                       +----------------------+
                                            ^
                                            |  
                                     +-----------+
                                     | T = t * G |
                                     +-----------+     
                                          ^                          
                                          |
                                         +---+   +----------------------------------------------------------+
                                         | t | = | TaggedHash ("Taptweak", P(local_delayed) || script_root) |
                                         +---+   +----------------------------------------------------------+
                                                                                                  ^
                                                                                                  |
                                                                                                +---+
                                                                                                | h |
                                                                                                +---+
                                                                                                  ^
                                                                                                  |
                                                                                           +--------------+
                                                                                           + OP_16 OP_CSV +
                                                                                           +--------------+

#### To Local Output

The to_local output is responsible for paying the local peer their channel balance. The output must be revocable by the remote party at all times and only after to_self_delay blocks should the local party be able to spend from the output.

As you can see in the diagram below, both these paths are added as Taproot leaves in the Taproot tree and a public NUMS (Nothing Up My Sleeve) point is used as the internal key which effectively cancels out the key-spend path.


    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| NUMS + T |
                       +----------+
                                ^
                                |
                          +-----------+        
                          | T = t * G |
                          +-----------+        
                                ^
                                |
     +---+   +----------------------------------------------+
     | t | = | TaggedHash ("Taptweak", NUMS || script_root) |
     +---+   +----------------------------------------------+
                                                    ^
                                                    |
                                                 +-----+
                          ---------------------> | hAB |<------------------
                         |                       +-----+                   |
                         |                                                 |
                         |                                                 |
                      +----+                                            +----+
                      | hA |                                            | hB |
                      +----+                                            +----+
                         ^                                                 ^
                         |                                                 |
      +------------------------------+                       +---------------------------+
      | P(local_delayed) OP_CHECKSIG |                       | P(local_delayed) OP_DROP  |
      |                              |                       |                           |
      | to_self_delay OP_CSV OP_DROP |                       | P(revocation) OP_CHECKSIG |
      +------------------------------+                       +---------------------------+

#### Key Derivations

Each commitment transaction uses unique keys: `localpubkey`, `local_htlcpubkey`, `remote_htlcpubkey`, `local_delayedpubkey`, and `remote_delayedpubkey`. These public keys are derived by simply adding a per-commitment point to their respective base points. As defined at Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G

```

- The `localpubkey` uses the local node's `payment_basepoint`;
- The `local_htlcpubkey` uses the local node's `htlc_basepoint`;
- The `remote_htlcpubkey` uses the remote node's `htlc_basepoint`;
- The `local_delayedpubkey` uses the local node's `delayed_payment_basepoint`;
- The `remote_delayedpubkey` uses the remote node's `delayed_payment_basepoint`.

The `revocationpubkey` is a blinded public key. When the local node prepares a new commitment transaction for the remote node, it derives the `revocationpubkey` by combining its own `revocation_basepoint` with the remote node’s `per_commitment_point`.

```
revocationpubkey = revocation_basepoint * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_point * SHA256(per_commitment_point || revocation_basepoint)
```

Later, when the remote node revokes that commitment by revealing the corresponding `per_commitment_secret`, the local node can derive the `revocationprivkey`. At that point, it possesses both secrets needed for the derivation: the `revocation_basepoint_secret` and the `per_commitment_secret`.

This construction ensures that neither the node providing the `basepoint` nor the node providing the `per_commitment_point` can derive the private key on their own—each requires the other’s secret.

```
revocationprivkey = revocation_basepoint_secret * SHA256(revocation_basepoint || per_commitment_point) + per_commitment_secret * SHA256(per_commitment_point || revocation_basepoint)
```

In [3]:
# Create Alice per-commitment
alice_per_commitment = per_commitment(alice_node_seed, commitment_number).get_compressed()
# Create Bob per-commitment
bob_per_commitment = per_commitment(bob_node_seed, commitment_number).get_compressed()

alice_delayed_key = derivated_key(alice_node_seed, 4)
alice_delayed_pubkey = alice_delayed_key.get_delayed_pubkey(alice_per_commitment)

print(f"Alice First Per Commitment Point: {alice_per_commitment.hex()}")

print(f"Alice Key: {alice_delayed_pubkey.get_bytes().hex()}")

Alice First Per Commitment Point: 02884eb80fe35012c1547a8d5a953147baa321b024f53a9aa83b3288793987e8a7
Alice Key: 03a621db343ed5c020cc8655fd742571e03e2512ed99ca84650fb35802859db3ab
